In [51]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
from scipy import stats

In [8]:
df = pd.read_csv('ab_data.csv')
df.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


In [9]:
# изучим данные
df.groupby(['group','landing_page']).count()
# видно, что часть юзеров из контрольной группы увидела новую страницу, часть тестируемой - старую
# удалим

user_id  timestamp  converted
group     landing_page                               
control   new_page         1928       1928       1928
          old_page       145274     145274     145274
treatment new_page       145311     145311     145311
          old_page         1965       1965       1965

In [27]:
data = df.query('(group == "control" & landing_page == "old_page") or (group == "treatment" & landing_page == "new_page")')

In [28]:
data.groupby(['group','landing_page']).count()

,,user_id,timestamp,converted
group,landing_page,,,
control,old_page,145274,145274,145274
treatment,new_page,145311,145311,145311


In [29]:
# посмотрим, есть ли дубли
data['user_id'].duplicated().sum()

1

In [31]:
data[data.duplicated(['user_id'],keep=False)]['user_id']

1899    773192
2893    773192
Name: user_id, dtype: int64

In [32]:
data = data.drop_duplicates(subset='user_id', keep="first")

In [34]:
# посчитаем конверсию
data.groupby(['group','landing_page','converted']).size()

group      landing_page  converted
control    old_page      0            127785
                         1             17489
treatment  new_page      0            128046
                         1             17264
dtype: int64

In [43]:
data.query('group == "control" & converted == 1') \
    .user_id.count() / data.query('group == "control"').user_id.count() * 100

12.03863045004612

In [44]:
data.query('group == "treatment" & converted == 1') \
    .user_id.count() / data.query('group == "treatment"').user_id.count() * 100

11.880806551510565

In [ ]:
# видим, что различия минимальны, однако они также могут быть статистически значимы

In [47]:
# подготовим данные
a = data[data['group'] == 'control']
b = data[data['group'] == 'treatment']

In [48]:
a_click = a.converted.sum()
a_noclick = a.converted.size - a.converted.sum()
b_click = b.converted.sum()
b_noclick = b.converted.size - b.converted.sum()

In [49]:
T = np.array([[a_click, a_noclick], [b_click, b_noclick]])

In [50]:
# проверим результат а/б теста с помощью хи-квадрата

In [52]:
print(scipy.stats.chi2_contingency(T,correction=False)[1])

0.18988337448194853


In [53]:
# p-value равен 0,19, это значит мы не можем отклонить нулевую гипотезу о равенстве средних:
# значимых различий не зафиксировали
# новую версию не раскатываем